# MNIST Classifier

In this notebook you will create both, an mnist tabular dataset and a classifier.

## 1.- import the Operating System (os) module in python and any other library you need

In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd

## 2.- As you can see each class has its own folder (Do it only for train). 

    - Iterate folder by folder ( os.listdir() )
    - Inside each folder: 
        1.- Read the image
        2.- Reshape it into a flat array (784,)
        3.- Save the data into a pandas dataframe apending the column name as the class
    - Save the data into a CSV

    Note: if it takes to long try doing only 100 images per folder and the teacher for the CSV.

In [28]:
# folder = np.arange(0, 10)
# img_csv = pd.DataFrame()
# for i in folder:
#     dir_path = f'/Users/felixschekerka/Desktop/Strive School/ai_jan22_felix/Chapter 02/11. Images/MNIST_digit_recognizer/trainingSet/trainingSet/{i}'
#     img_dir = os.listdir(dir_path)
#     for file in img_dir:
#         img = Image.open(dir_path + '/' + file)
#         arr = np.array(img, dtype=int)
#         arr = arr.flatten()
#         img_csv[f'{file}'] = arr
# img_csv = img_csv.T
# img_csv.to_csv('img_csv')


parent_dir = "/Users/felixschekerka/Desktop/Strive School/ai_jan22_felix/Chapter 02/11. Images/MNIST_digit_recognizer/trainingSet/trainingSet"
data = pd.DataFrame()
for folder in os.listdir(parent_dir):
    print(folder)
    if folder == "trainingSet":
        for f in os.listdir(parent_dir+"/"+folder):
            print(f)
            for k in os.listdir(parent_dir+"/"+folder + "/" + f):
                print(k)
                class_data = np.zeros(  ( len(os.listdir(parent_dir+"/"+folder+"/"+f+"/"+k) ), 785) )
                print(class_data.shape)
                for i, img_name in enumerate(os.listdir(parent_dir+"/"+folder+"/"+f +"/"+k)):
                    #print(i)
                    img = Image.open(parent_dir+"/"+folder+"/"+f+"/"+k+"/"+img_name)
                    img_arr = np.array(img, dtype=int)
                    img_arr = img_arr.flatten()
                #print(img_arr.shape)
                    class_data[i,:784] = img_arr
                    class_data[i,784] = int(k)
                class_data = pd.DataFrame(class_data)
                data = pd.concat([data, class_data])
data.to_csv("train.csv", index=False)



.DS_Store
9
0
7
6
1
8
4
3
2
5


## 3.- Load the CSV

In [14]:
images_df = pd.read_csv('img_csv', index_col=0)
images_df

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
img_13831.jpg,0,0,0,0,0,0,0,0,0,6,...,0,0,0,0,0,0,0,0,0,0
img_14186.jpg,0,0,0,0,0,0,0,0,9,2,...,0,0,0,0,0,0,0,0,0,0
img_30030.jpg,0,0,0,0,0,0,0,0,0,19,...,0,0,1,0,1,6,0,0,0,0
img_21129.jpg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
img_37993.jpg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
img_35623.jpg,0,0,0,0,0,0,0,0,3,0,...,3,0,0,0,0,0,0,0,0,0
img_21130.jpg,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
img_28672.jpg,0,0,0,0,0,0,0,0,1,6,...,3,0,0,3,4,0,0,0,0,0
img_13828.jpg,0,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0


## 4.- Create a dictionary of models (No preprocessing needed, it has already been done).
    
    Include both, tree models and mult models.

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import model_selection 
from sklearn import metrics  
import time
import sklearn

In [38]:
models = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(n_estimators=10),
  "Random Forest": RandomForestClassifier(n_estimators=100),
  "AdaBoost": AdaBoostClassifier(n_estimators=100),
  "Skl GBM": GradientBoostingClassifier(n_estimators=100),
  "Skl HistGBM": HistGradientBoostingClassifier(max_iter=100),
  "XGBoost": XGBClassifier(n_estimators=100),
  "LightGBM": LGBMClassifier(n_estimators=100),
  "CatBoost": CatBoostClassifier(n_estimators=100)}

## 5.- Using either cross validation or stratification find out which is the best model
    - Base your code on the previous two days examples

In [43]:
x = images_df.values[:,:783]
y = images_df.values[:,783]

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=0)

res = pd.DataFrame({'Model': [], 'Acc': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in models.items():
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    pred = model.predict(x_val)

    res = res.append({'Model': model_name,
                        'Acc': metrics.accuracy_score(y_val, pred)*100,
                        'Bal Acc': metrics.balanced_accuracy_score(y_val, pred)*100,
                        'Time': total_time},
                        ignore_index=True)

results_ord = res.sort_values(by=['Acc'], ascending=False, ignore_index=True)
results_ord.index += 1
print(results_ord)



ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

## Optional: Can you rotate an image?